In [1]:
import numpy as np 
import pandas as pd
data= pd.read_csv("/Users/vanessaguerrier/Downloads/projet_TER/ner_dataset.csv",encoding="ISO-8859-1")

In [4]:
nb_sentence= data["Sentence #"].unique()
len(nb_sentence)

# id_begin_sentences = [data.index[data["Sentence #"] == x][0] for x in nb_sentence if pd.notna(x)]
# id_begin_sentences

47960

In [6]:
X= data["Word"]
y= data["Tag"]
id2label = {i: label for i, label in enumerate(y)}

In [11]:


from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
labels = y.unique()
num_labels = len(y.unique())

model = AutoModel.from_pretrained("bert-base-cased")

In [12]:
import torch
from torch import nn,optim
import torch.nn.functional as F
import pytorch_lightning as pl
class NER (pl.LightningModule):
    
    def __init__(self, ndim=278,nb_target=10):
        super().__init__()
        self.linear1 = nn.Linear(ndim, 90)
        self.relu1 = nn.ReLU() 
        self.linear2 = nn.Linear(90, nb_target)
        self.relu2 = nn.ReLU()
        self.droupOut = nn.Dropout(0.30) 
        
    def forward(self, x1):
        x = self.linear1(x1)
        x = self.relu1(x)
        x = self.droupOut(x)
        x = self.linear2(x)
        x = self.relu2(x)
        return F.log_softmax(x, dim=1)  # softmax pour classification multi-classes

    def training_step(self, batch, batch_idx):
        x1, y = batch
        logits = self(x1)
        loss = F.nll_loss(logits, y)
        self.log("train_loss", loss)
        return loss
    
    def validation_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.nll_loss(logits, y)
        preds = logits.argmax(dim=1)
        acc = (preds == y).float().mean()
        self.log("val_loss", loss, prog_bar=True)
        self.log("val_acc", acc, prog_bar=True)
    
    def test_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.nll_loss(logits, y)
        preds = logits.argmax(dim=1)
        acc = (preds == y).float().mean()
        self.log("test_loss", loss)
        self.log("test_acc", acc)

        
    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=1e-3)

In [17]:
text = "The model was trained for 10 epochs on 4 GPUs."
inputs = tokenizer(text, return_tensors="pt")
print(inputs)
with torch.no_grad():
    outputs = model(**inputs)

# Shape: (batch, tokens, hidden_size)
token_embeddings = outputs.last_hidden_state

print(token_embeddings.shape)


{'input_ids': tensor([[  101,  1109,  2235,  1108,  3972,  1111,  1275,   174,  5674, 17704,
          1113,   125, 15175,  2591,  1116,   119,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}
torch.Size([1, 17, 768])
